TensorFlow变量是表示程序操作的共享和persistent state的最佳方式。 

变量通过tf.Variable类来操作。 tf.Variable表示张量，通过运行op可以改变它的值。与tf.Tensor对象不同，tf.Variable存在于单个session.run调用的上下文之外。

在内部，一个tf.Variable存储一个persistent张量。具体操作允许您读取和修改张量的值。这些修改在多个tf.Sessions中可见，因此多个worker可以看到tf.Variable的相同值。

## 创建变量

创建变量的最好方法是调用`tf.get_variable`函数。该函数要求您指定变量的名称。此名称将被其他副本用于访问相同变量，以及在检查点和导出模型时命名此变量的值。 tf.get_variable还允许您重复使用先前创建的同名变量，从而轻松定义重用图层的模型。

要使用tf.get_variable创建变量，只需提供名称和形状

```python
my_variable = tf.get_variable("my_variable", [1, 2, 3])
```

这将创建一个名为“my_variable”的变量，它是一个**形状为[1,2,3]的三维张量**。默认情况下，该变量具有dtype tf.float32，其初始值将通过tf.glorot_uniform_initializer进行随机化。

您可以选择将dtype和初始化程序指定为tf.get_variable。例如：

```python
my_int_variable = tf.get_variable("my_int_variable", [1, 2, 3], dtype=tf.int32,
  initializer=tf.zeros_initializer)
```

TensorFlow提供了许多方便的初始化程序。或者，您可以用一个tf.Tensor的值初始化另一个tf.Variable以。例如：

```python
other_variable = tf.get_variable("other_variable", dtype=tf.int32,
  initializer=tf.constant([23, 42]))
```

请注意，当初始值设定项是tf.Tensor时，不应指定变量的形状，因为初始化张量的形状将被使用。

## Variable collections

由于TensorFlow程序的不连接部分可能需要创建变量，因此使用单一方法访问所有变量有时很有用。出于这个原因，TensorFlow提供集合，这些集合被命名为张量列表或其他对象，如tf.Variable实例。

默认情况下，每个tf.Variable被放置在以下两个集合中：

- tf.GraphKeys.GLOBAL_VARIABLES - 可以在多个设备上共享的变量.
- tf.GraphKeys.TRAINABLE_VARIABLES - TensorFlow用来计算梯度的变量.

如果您不想让变量可训练，请将其添加到tf.GraphKeys.LOCAL_VARIABLES集合中。例如，以下代码片段演示了如何将名为my_local的变量添加到此集合中：

```python
my_local = tf.get_variable("my_local", shape=(),
collections=[tf.GraphKeys.LOCAL_VARIABLES])
```

或者，您可以指定trainable = False作为tf.get_variable的参数：

```python
my_non_trainable = tf.get_variable("my_non_trainable",
                                   shape=(),
                                   trainable=False)
```

您也可以使用自己的collection。**任何字符串都是有效的集合名称，并且不需要显式创建collection**。要在创建变量后向collection添加变量（或任何其他对象），请调用`tf.add_to_collection`。例如，以下代码将名为my_local的现有变量添加到名为my_collection_name的集合中：

```python
tf.add_to_collection("my_collection_name", my_local)
```

并且可以检索已放置在集合中的所有变量（或其他对象）的列表，您可以使用：

```python
tf.get_collection("my_collection_name")
```

### Device placement

就像任何其他TensorFlow操作一样，您可以在特定设备上放置变量。例如，以下片段创建一个名为v的变量并将其放置在第二个GPU设备上：

```python
with tf.device("/device:GPU:1"):
  v = tf.get_variable("v", [1])
```

变量在分布式设置中处于正确的设备中尤其重要。例如，不小心将变量放在工作者而不是参数服务器上，可能会严重减慢培训速度，或者在最坏的情况下，让每个worker都有自己的独立副本。出于这个原因，我们提供了`tf.train.replica_device_setter`，它可以自动将参数放置在参数服务器中。例如：

```python
cluster_spec = {
    "ps": ["ps0:2222", "ps1:2222"],
    "worker": ["worker0:2222", "worker1:2222", "worker2:2222"]}
with tf.device(tf.train.replica_device_setter(cluster=cluster_spec)):
  v = tf.get_variable("v", shape=[20, 20])  # this variable is placed
                                            # in the parameter server
                                            # by the replica_device_setter
```


## Initializing variables

在你使用变量之前，它必须被初始化。如果您在low-level的TensorFlow API中进行编程（即，您正在显式创建自己的图形和会话），则必须明确地初始化变量。在训练模型之前，大多数高级框架（如tf.contrib.slim，tf.estimator.Estimator和Keras）会自动为您初始化变量。

显式初始化在其他方面很有用，因为它允许您在从检查点重新加载模型时不会重新运行潜在的昂贵初始化程序，并且在分布式设置中共享随机初始化变量时允许确定性。

要在训练开始前一次初始化所有可训练变量，请调用`tf.global_variables_initializer（）`。该函数返回一个负责初始化`tf.GraphKeys.GLOBAL_VARIABLES`集合中所有变量的操作。运行此操作会初始化所有变量。例如：

```python
session.run(tf.global_variables_initializer())
# Now all variables are initialized.
```

如果你确实需要自己初始化变量，你可以运行变量的初始化操作。例如：

```python
session.run(my_variable.initializer)
```

**您还可以询问哪些变量尚未初始化**。例如，下面的代码打印所有尚未初始化的变量的名称：

```python
print(session.run(tf.report_uninitialized_variables()))
```

> 请注意，默认`tf.global_variables_initializer`不指定变量初始化的顺序。因此，**如果变量的初始值取决于另一个变量的值，那么很可能会出现错误**。任何时候，如果在并非所有变量都被初始化的环境中使用变量的值（例如，如果在初始化另一个变量时使用变量的值），最好使用`variable.initialized_value（）`而不是`variable`：

```python
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
w = tf.get_variable("w", initializer=v.initialized_value() + 1)
```

## Using variables

要在TensorFlow图中使用tf.Variable的值，只需将其作为普通的tf.Tensor对待即可：

```python
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
w = v + 1  # w is a tf.Tensor which is computed based on the value of v.
           # Any time a variable is used in an expression it gets automatically
           # converted to a tf.Tensor representing its value.
```

要为变量赋值，请使用tf.Variable类中的assign，assign_add和friends方法。例如，以下是您可以如何调用这些方法：

```python
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
assignment = v.assign_add(1)
tf.global_variables_initializer().run()
sess.run(assignment)  # or assignment.op.run(), or assignment.eval()
```

大多数TensorFlow优化器都有专门的操作，根据某种梯度下降算法有效地更新变量的值。有关如何使用优化器的说明，请参阅tf.train.Optimizer。

由于变量是可变的，因此知道在任何时间点使用的变量值是什么版本有时很有用。要在发生某事后强制重新读取变量的值，可以使用tf.Variable.read_value。例如：

```python
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
assignment = v.assign_add(1)
with tf.control_dependencies([assignment]):
  w = v.read_value()  # w is guaranteed to reflect v's value after the
                      # assign_add operation.
```

## Sharing variables

TensorFlow支持两种共享变量的方式：

- 显式传递tf.Variable对象.
- 在tf.variable_scope对象中隐式包装tf.Variable对象。

虽然显式传递变量的代码非常清晰，但编写TensorFlow函数有时很方便，它们在其实现中隐式使用变量。 tf.layer的大部分功能层都使用这种方法，以及所有的tf.metrics和其他一些库实用程序。

变量作用域允许您在调用隐式创建和使用变量的函数时控制变量重用。它们还允许您以分层和可理解的方式命名变量。

例如，假设我们编写一个函数来创建一个convolutional / relu层：

```python
def conv_relu(input, kernel_shape, bias_shape):
    # Create variable named "weights".
    weights = tf.get_variable("weights", kernel_shape,
        initializer=tf.random_normal_initializer())
    # Create variable named "biases".
    biases = tf.get_variable("biases", bias_shape,
        initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input, weights,
        strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)
```

此功能使用short names为weights, biases，这很清晰。然而，在一个真实的模型中，我们需要很多这样的卷积图层，并且反复调用这个函数将不起作用：

```python
input1 = tf.random_normal([1,10,10,32])
input2 = tf.random_normal([1,20,20,32])
x = conv_relu(input1, kernel_shape=[5, 5, 32, 32], bias_shape=[32])
x = conv_relu(x, kernel_shape=[5, 5, 32, 32], bias_shape = [32])  # This fails.
```

由于期望的行为不清楚（创建新的变量或重新使用现有的变量？）TensorFlow将失败。然而，在不同范围内调用conv_relu会澄清我们想要创建新变量：

```python
def my_image_filter(input_images):
    with tf.variable_scope("conv1"):
        # Variables created here will be named "conv1/weights", "conv1/biases".
        relu1 = conv_relu(input_images, [5, 5, 32, 32], [32])
    with tf.variable_scope("conv2"):
        # Variables created here will be named "conv2/weights", "conv2/biases".
        return conv_relu(relu1, [5, 5, 32, 32], [32])
```

如果你想要共享变量，你有两个选择。首先，您可以使用reuse = True来创建具有相同名称的作用域：

```python
with tf.variable_scope("model"):
  output1 = my_image_filter(input1)
with tf.variable_scope("model", reuse=True):
  output2 = my_image_filter(input2)
```

你也可以调用scope.reuse_variables（）来触发重用：

```python
with tf.variable_scope("model") as scope:
  output1 = my_image_filter(input1)
  scope.reuse_variables()
  output2 = my_image_filter(input2)
```

由于取决于范围的确切字符串名称可能会感到危险，因此也可以基于另一个范围初始化变量作用域

```python
with tf.variable_scope("model") as scope:
  output1 = my_image_filter(input1)
with tf.variable_scope(scope, reuse=True):
  output2 = my_image_filter(input2)
```